In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
#Load City Data from part 1
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,17.97,68,0,5.66,US,1715194835
1,1,berezovyy,51.6667,135.7000,10.73,55,100,2.29,RU,1715194836
2,2,tiksi,71.6872,128.8694,-9.37,95,63,3.24,RU,1715194836
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.32,67,35,10.70,SH,1715194836
4,4,hamilton,39.1834,-84.5333,29.79,51,0,2.06,US,1715194369


In [3]:
#create a map to display each city within the dataframe
cities_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City"
)
cities_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

In [7]:
# Ideal weather locations
ideal_df = city_data_df.loc[(city_data_df['Max Temp']< 20) & (city_data_df['Max Temp'] >10) & (city_data_df['Cloudiness']<40) & (city_data_df['Cloudiness']>0) & (city_data_df['Wind Speed']< 10) & (city_data_df['Lat']<0)]
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,hluti,-27.2167,31.6167,17.10,92,28,0.23,SZ,1715194840
166,166,kerikeri,-35.2268,173.9474,10.51,83,5,3.58,NZ,1715194855
178,178,eaglehawk,-36.7333,144.2500,12.05,67,14,2.41,AU,1715194856
198,198,queenstown,-31.8976,26.8753,13.59,81,4,2.17,ZA,1715194858
237,237,lubango,-14.9172,13.4925,19.92,64,20,1.03,AO,1715194863
331,331,broken hill,-31.9500,141.4333,13.14,74,12,4.39,AU,1715194874
337,337,danielskuil,-28.1887,23.5395,19.31,30,4,3.68,ZA,1715194874
368,368,talcahuano,-36.7167,-73.1167,10.82,76,20,3.60,CL,1715194623
404,404,port macquarie,-31.4333,152.9167,14.38,79,20,1.20,AU,1715194882
441,441,concepcion,-36.8270,-73.0498,10.89,76,20,3.60,CL,1715194886


In [8]:
#create new dataframe for hotel search
hotel_df = pd.DataFrame(ideal_df[['City','Lat','Lng', 'Country','Humidity']])
hotel_df.head()

,City,Lat,Lng,Country,Humidity
44,hluti,-27.2167,31.6167,SZ,92
166,kerikeri,-35.2268,173.9474,NZ,83
178,eaglehawk,-36.7333,144.2500,AU,67
198,queenstown,-31.8976,26.8753,ZA,81
237,lubango,-14.9172,13.4925,AO,64


In [9]:
#Hotel Search

# Set parameters to search for hotels
radius = 100000
params = {
    "categories":"accommodation",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to start hotel search
print("Starting hotel search")

# Iterate through the hotel_df dataframe
for index, row in hotel_df.iterrows():
    
    # Get the city name
    city = row["City"]
    
    # Get latitude, longitude from the dataframe
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
           
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()

    
    # take first hotel from the results and store in the DataFrame
    try:
        hotel_df.loc[index, "Hotel"] = response['features'][0]['properties']['name']
        hotel_df.loc[index, "Country"] = response['features'][0]['properties']['country']
        
    except (KeyError, IndexError):
        # If no hotel is found, log the error.
        hotel_df.loc[index, "Hotel"] = "None"
        
    # Log the search results
    print(f"nearest hotel from {city}: {hotel_df.loc[index, 'Hotel']}")

# Display sample data
hotel_df

Starting hotel search
nearest hotel from hluti: Coco Cabana
nearest hotel from kerikeri: Kerikeri Homestead Motel
nearest hotel from eaglehawk: None
nearest hotel from queenstown: Roche House
nearest hotel from lubango: Novo Hotel
nearest hotel from broken hill: Sturt Motel
nearest hotel from danielskuil: None
nearest hotel from talcahuano: Club Recreativo Armada de Chile
nearest hotel from port macquarie: Town Beach Motor Inn
nearest hotel from concepcion: Hotel One (ex-Biobio)
nearest hotel from oranjemund: Oryx guesthouse


,City,Lat,Lng,Country,Humidity,Hotel
44,hluti,-27.2167,31.6167,South Africa,92,Coco Cabana
166,kerikeri,-35.2268,173.9474,New Zealand,83,Kerikeri Homestead Motel
178,eaglehawk,-36.7333,144.2500,AU,67,None
198,queenstown,-31.8976,26.8753,South Africa,81,Roche House
237,lubango,-14.9172,13.4925,Angola,64,Novo Hotel
331,broken hill,-31.9500,141.4333,Australia,74,Sturt Motel
337,danielskuil,-28.1887,23.5395,ZA,30,None
368,talcahuano,-36.7167,-73.1167,Chile,76,Club Recreativo Armada de Chile
404,port macquarie,-31.4333,152.9167,Australia,79,Town Beach Motor Inn
441,concepcion,-36.8270,-73.0498,Chile,76,Hotel One (ex-Biobio)


In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   tiles = "OSM",
                                   frame_width = 700,
                                   frame_height = 500,
                                   size = "Humidity",
                                   hover_cols = ["Hotel","Country","Humidity"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel,Country)